In [13]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import plotly.graph_objects as go
import plotly.subplots as sp
import plotly.express as px
import pandas as pd
import seaborn as sns

In [14]:
df = pd.read_csv('./data/World Energy Consumption.csv')

In [23]:
from src.preprocessing.renaming_countries import *
from src.preprocessing.insert_iso_codes import get_iso_code
from src.preprocessing.consumptions_dataset import retain_consumptions
rename_countries(df)
df = drop_countries(df)
get_iso_code(df)
df  = retain_consumptions(df)

In [24]:
filtered_list = [s for s in list(df.columns) if 'elec_per_capita' in s.lower()]
strings_to_remove = [ 'other_renewables_elec_per_capita_exc_biofuel']
filtered_list = list(filter(lambda item: item not in strings_to_remove,filtered_list))
formatted_sources = [' '.join(word.capitalize() for word in item.split('_')) for item in filtered_list]

In [33]:
# Identify the latest year in the dataset
latest_year = df['year'].max()

# Filter the dataset for the latest year and create an explicit copy
latest_data = df[df['year'] == latest_year].copy()

# Define the classification of energy types
metric_groups = {
    'biofuel_consumption': 'renewables',
    'solar_consumption': 'renewables',
    'wind_consumption': 'renewables',
    'other_renewable_consumption': 'renewables',
    'renewables_consumption': 'renewables',
    'hydro_consumption': 'renewables',
    'coal_consumption': 'not_renewables',
    'fossil_fuel_consumption': 'not_renewables',
    'gas_consumption': 'not_renewables',
    'oil_consumption': 'not_renewables',
    'low_carbon_consumption': 'not_renewables',
    'nuclear_consumption': 'nuclear'
}

# Calculate sums for renewable and non-renewable energy consumption
latest_data['renewable_energy'] = latest_data[[col for col in metric_groups if metric_groups[col] == 'renewables']].sum(axis=1)
latest_data['non_renewable_energy'] = latest_data[[col for col in metric_groups if metric_groups[col] == 'not_renewables']].sum(axis=1)

# Continue with finding the top and bottom countries and plotting

# Sum up renewable and non-renewable consumptions
latest_data['renewable_energy'] = latest_data[[col for col in metric_groups if metric_groups[col] == 'renewables']].sum(axis=1)
latest_data['non_renewable_energy'] = latest_data[[col for col in metric_groups if metric_groups[col] == 'not_renewables']].sum(axis=1)

# Find the top 5 and bottom 5 countries
top_5_renewables = latest_data.nlargest(5, 'renewable_energy')
bottom_5_non_renewables = latest_data.nlargest(5, 'non_renewable_energy')


In [43]:
import plotly.graph_objects as go
import pandas as pd


# Initialize the figure
fig = go.Figure()

# Add traces for Renewable and Non-Renewable Energy
for df, label, color in zip([top_5_renewables, bottom_5_non_renewables], 
                            ['Renewable Energy', 'Non Renewable Energy'], 
                            ['green', 'red']):
    fig.add_trace(go.Scatter(
        x=df['country'],  # Ensure this is directly using the 'country' column
        y=df[label.lower().replace(" ", "_")],  # Accessing columns dynamically
        mode='markers',
        marker=dict(
            size=df[label.lower().replace(" ", "_")] / df[label.lower().replace(" ", "_")].max() * 100,  # Size normalization
            color=color
        ),
        name=label
    ))

fig.update_layout(
    title='Prospectives on Energy Consumption - Top 5 consumer of Renewables and Not-Renewables | 2022',
    xaxis=dict(
        title='Country',
        type='category'  # Ensuring the x-axis is treated as categorical data
    ),
    yaxis=dict(
        title='Cumulative Energy Consumption -Kw per Capita  '
    ),
    legend_title="Energy Type",
)

# Save the figure to an HTML file
fig.write_html('./outputs/prospectives.html')
print("Plot saved as an interactive HTML file at 'energy_consumption_comparison.html'.")


Plot saved as an interactive HTML file at 'energy_consumption_comparison.html'.
